In [1]:
import pandas as pd
import numpy as np
import random
import locale


In [2]:
# Criando as datas de 01/01/2024 até 11/06/2025
datas = pd.date_range(start='2024-01-01', end='2025-06-11', freq='D')

# Criando um DataFrame com as datas
df = pd.DataFrame({'data': datas})

# Define o locale para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # No Windows pode ser 'Portuguese_Brazil.1252'

# Gera o nome do dia da semana em português e pega os 3 primeiros caracteres
df['dia_semana'] = df['data'].dt.strftime('%A').str[:3].str.lower()

# Filtrando apenas dias úteis (segunda a sexta)
dias_uteis = df[df['dia_semana'].isin(['seg', 'ter', 'qua', 'qui', 'sex'])]

# Média de vendas nos dias úteis
amostra_vendas = np.random.randint(29, 58, size=len(dias_uteis))
media_uteis = int(np.mean(amostra_vendas))

# Função para gerar valores aleatórios de produto, oferta, valor e promo
def gera_valores_aleatorios():
    # Nome do produto
    letras = ['A', 'B', 'C', 'D']
    produto = random.choice(letras)

    # Gerando valores aleatórios 'S' ou 'N' para a coluna 'oferta'
    oferta = random.choice(['S', 'N'])

    # Mapeando os valores dos produtos
    valores = {'A': 129.90, 'B': 119.90, 'C': 109.90, 'D': 99.90}
    valor = valores[produto]

    # Lista de promoções possíveis
    promocoes = ['mensal', '30_P', '50_P']

    

    # Função para aplicar a lógica de promoção
    def atribui_promocao(oferta):
        if oferta == 'S':
            return random.choice(promocoes)
        else:
            return 'Sem_Oferta'

    # Atribuindo promoção
    promo = atribui_promocao(oferta)

    return produto, oferta, valor, promo

# Função para definir quantas vendas por dia
def num_vendas(dia):
    if dia == 'dom':
        return 0
    elif dia == 'sab':
        return max(1, media_uteis // 2)
    else:
        return random.randint(29, 57)
    

# Gerando vendas para cada linha, com valores aleatórios
df_vendas = df.loc[df.index.repeat(df['dia_semana'].apply(num_vendas))].reset_index(drop=True)

# Gerando valores aleatórios para cada linha de venda
df_vendas['produto'], df_vendas['oferta'], df_vendas['valor'], df_vendas['promo'] = zip(*[gera_valores_aleatorios() for _ in range(len(df_vendas))])

df_vendas['ano_mes'] = df_vendas['data'].dt.strftime('%Y%m').astype(int)



# Exibindo uma amostra das vendas geradas
print(df_vendas.head())

        data dia_semana produto oferta  valor       promo  ano_mes
0 2024-01-01        seg       C      N  109.9  Sem_Oferta   202401
1 2024-01-01        seg       D      S   99.9        50_P   202401
2 2024-01-01        seg       B      S  119.9        30_P   202401
3 2024-01-01        seg       B      N  119.9  Sem_Oferta   202401
4 2024-01-01        seg       B      N  119.9  Sem_Oferta   202401


In [3]:
# Exibindo as primeiras linhas do DataFrame
df_vendas

,data,dia_semana,produto,oferta,valor,promo,ano_mes
0,2024-01-01,seg,C,N,109.9,Sem_Oferta,202401
1,2024-01-01,seg,D,S,99.9,50_P,202401
2,2024-01-01,seg,B,S,119.9,30_P,202401
3,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401
4,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401
...,...,...,...,...,...,...,...
19282,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506
19283,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506
19284,2025-06-11,qua,D,S,99.9,50_P,202506
19285,2025-06-11,qua,C,N,109.9,Sem_Oferta,202506


In [4]:
# Criando a Vigencia

df_vendas['vigencia'] = df_vendas['promo'].apply(lambda x: 12 if x == 'mensal' else (0 if x == 'Sem_Oferta' else 3))

# Exibindo o DataFrame para verificar
print(df_vendas.head())


        data dia_semana produto oferta  valor       promo  ano_mes  vigencia
0 2024-01-01        seg       C      N  109.9  Sem_Oferta   202401         0
1 2024-01-01        seg       D      S   99.9        50_P   202401         3
2 2024-01-01        seg       B      S  119.9        30_P   202401         3
3 2024-01-01        seg       B      N  119.9  Sem_Oferta   202401         0
4 2024-01-01        seg       B      N  119.9  Sem_Oferta   202401         0


In [5]:
# Criando valor da oferta
df_vendas['oferta_vl'] = df_vendas.apply(
    lambda row: row['valor'] - 12 if row['promo'] == 'mensal' else
                row['valor'] * 0.7 if row['promo'] == '30_P' else
                row['valor'] * 0.5 if row['promo'] == '50_P' else
                row['valor'],
    axis=1
)

# Exibindo o DataFrame para verificar
df_vendas

# data |	produto	| oferta	| valor	| promo	| vigencia	| oferta_vl	| vigencia_ctt


,data,dia_semana,produto,oferta,valor,promo,ano_mes,vigencia,oferta_vl
0,2024-01-01,seg,C,N,109.9,Sem_Oferta,202401,0,109.90
1,2024-01-01,seg,D,S,99.9,50_P,202401,3,49.95
2,2024-01-01,seg,B,S,119.9,30_P,202401,3,83.93
3,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90
4,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90
...,...,...,...,...,...,...,...,...,...
19282,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506,0,129.90
19283,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506,0,129.90
19284,2025-06-11,qua,D,S,99.9,50_P,202506,3,49.95
19285,2025-06-11,qua,C,N,109.9,Sem_Oferta,202506,0,109.90


In [6]:
# Criando o Ticket Médio
# Calculando o ticket médio com base na vigência e no valor da oferta

df_vendas['ticket_medio'] = df_vendas.apply(
    lambda row: round((3 * row['oferta_vl'] + 9 * row['valor']) / 12, 2) if row['vigencia'] == 3 else
                round(row['oferta_vl'], 2),
    axis=1
)

In [7]:
df_vendas

,data,dia_semana,produto,oferta,valor,promo,ano_mes,vigencia,oferta_vl,ticket_medio
0,2024-01-01,seg,C,N,109.9,Sem_Oferta,202401,0,109.90,109.90
1,2024-01-01,seg,D,S,99.9,50_P,202401,3,49.95,87.41
2,2024-01-01,seg,B,S,119.9,30_P,202401,3,83.93,110.91
3,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90,119.90
4,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90,119.90
...,...,...,...,...,...,...,...,...,...,...
19282,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506,0,129.90,129.90
19283,2025-06-11,qua,A,N,129.9,Sem_Oferta,202506,0,129.90,129.90
19284,2025-06-11,qua,D,S,99.9,50_P,202506,3,49.95,87.41
19285,2025-06-11,qua,C,N,109.9,Sem_Oferta,202506,0,109.90,109.90


In [8]:
import pandas as pd
import random

# Supondo que você já tenha df_vendas carregado:
# df_vendas = pd.read_csv("sua_base.csv")

# Lista de gerentes
gerentes = ["Carlos Souza", "Fernanda Lima", "João Pereira", "Mariana Alves", "Ricardo Rocha", "Patrícia Costa"]

# Gerar 36 vendedores únicos
nomes = ["Ana", "Bruno", "Camila", "Daniel", "Eduarda", "Felipe", "Giovana", "Henrique", "Isabela", "José", "Karen", "Lucas",
         "Marina", "Nicolas", "Olívia", "Pedro", "Quésia", "Rafael", "Sabrina", "Tiago", "Ursula", "Victor", "Wesley", "Ximena",
         "Yuri", "Zuleica", "Amanda", "Caio", "Débora", "Elias", "Fabiana", "Gabriel", "Heitor", "Ingrid", "Jonas", "Larissa"]
sobrenomes = ["Silva", "Santos", "Oliveira", "Souza", "Lima", "Costa"]

# Garantir que os nomes sejam únicos
vendedores_unicos = list(set(f"{random.choice(nomes)} {random.choice(sobrenomes)}" for _ in range(100)))
vendedores = vendedores_unicos[:36]  # Pegar os 36 primeiros únicos

# Mapear 6 vendedores para cada gerente
mapa_gerente_vendedores = {
    gerente: vendedores[i * 6:(i + 1) * 6] for i, gerente in enumerate(gerentes)
}

# Criar DataFrame da hierarquia
relacao_vendedor_gerente = []
for gerente, vends in mapa_gerente_vendedores.items():
    for vendedor in vends:
        relacao_vendedor_gerente.append({"vendedor": vendedor, "Gerente": gerente})

df_hierarquia = pd.DataFrame(relacao_vendedor_gerente)

# Gerar IDs únicos para os vendedores
df_hierarquia["id_vendedor"] = [f"V{str(i+1).zfill(3)}" for i in range(len(df_hierarquia))]

# Atribuir vendedores ao df_vendas com base no sorteio dos vendedores da hierarquia
# Primeiro sorteamos vendedores e usamos a hierarquia para trazer o gerente correto
df_vendas["vendedor"] = random.choices(df_hierarquia["vendedor"].tolist(), k=len(df_vendas))

# Merge para trazer o gerente e o id_vendedor corretos
df_vendas = df_vendas.merge(df_hierarquia, on="vendedor", how="left")

# Validar se algum vendedor ficou com mais de um gerente (não deveria acontecer)
validacao = df_vendas.groupby("vendedor")["Gerente"].nunique()
vendedores_invalidos = validacao[validacao > 1]

if not vendedores_invalidos.empty:
    print("⚠️ Vendedores com mais de um gerente encontrados:")
    print(vendedores_invalidos)
else:
    print("✅ Todos os vendedores estão corretamente associados a apenas um gerente.")


✅ Todos os vendedores estão corretamente associados a apenas um gerente.


In [9]:
df_vendas.head()

,data,dia_semana,produto,oferta,valor,promo,ano_mes,vigencia,oferta_vl,ticket_medio,vendedor,Gerente,id_vendedor
0,2024-01-01,seg,C,N,109.9,Sem_Oferta,202401,0,109.90,109.90,Olívia Costa,Patrícia Costa,V035
1,2024-01-01,seg,D,S,99.9,50_P,202401,3,49.95,87.41,Larissa Souza,Mariana Alves,V019
2,2024-01-01,seg,B,S,119.9,30_P,202401,3,83.93,110.91,Giovana Santos,João Pereira,V014
3,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90,119.90,Heitor Oliveira,Mariana Alves,V024
4,2024-01-01,seg,B,N,119.9,Sem_Oferta,202401,0,119.90,119.90,Rafael Souza,Patrícia Costa,V033


In [24]:
df_vendas['data'] = df_vendas['data'].astype(str)
